# 1. XLang v1 — Minimal Excel Compiler Notebook

**Project:** Output Representation Optimisation (ORO)  
**Version:** 1.0 (Preview)  
**Date:** 2025-11-17  
**Author:** Jackson Chai

---

## 1.1 Purpose of This Notebook

This notebook implements a minimal working version of XLang v1, a concise markup language designed for large language models (LLMs) to generate Excel workbooks directly.

The main objectives are to:

1. Understand the basic structure of XLang.  
2. Build a minimal compiler in Python.  
3. Test the compilation end to end using a simple example.  
4. Provide a foundation for extending XLang with styles, merges, sequences and patterns.

---

## 1.2 Scope of This Version

In this first version of the notebook, we only support the core tags:

1. xworkbook  
2. xsheet  
3. xrow  
4. xv  
5. xcell  

Support for additional tags such as xstyle, xmerge, xseq, xplace, xrepeat, xpattern and xfill will be added later, after we verify that the core pipeline works correctly.


In [1]:
# ============================================================
# 1. Imports and basic setup
# ============================================================

# Uncomment the following line if your environment does not have openpyxl installed.
# %pip install openpyxl

from xml.etree import ElementTree as ET
from openpyxl import Workbook, load_workbook
from pathlib import Path

# 2. Column Letter Conversion Helper

## 2.1 Motivation

Excel columns are labelled using letters such as A, B, C, …, Z, AA, AB and so on.  
To place values programmatically, we must convert these letters into numeric indices.

Examples:

- A becomes 1  
- B becomes 2  
- Z becomes 26  
- AA becomes 27  

The following helper function handles this conversion in a robust way.


In [2]:
# ============================================================
# 2. Column letter to index helper
# ============================================================

def col_letter_to_index(col: str) -> int:
    """
    Convert Excel column letters (A, B, Z, AA, AB etc.)
    into a 1-based integer column index.
    """
    col = col.strip().upper()
    result = 0

    for ch in col:
        if not ("A" <= ch <= "Z"):
            raise ValueError(f"Invalid column letter: {col}")
        result = result * 26 + (ord(ch) - ord("A") + 1)

    return result

# 3. Minimal XLang Validator

## 3.1 Why Validate

Before compiling XLang into Excel, it is useful to catch simple structural errors early.  
A minimal validator helps ensure that the core tags are used correctly and prevents confusing runtime failures.

## 3.2 Validation Rules Implemented

For this initial version, the validator checks the following:

1. The root tag must be `xworkbook`.  
2. Each `xsheet` must have a `name` attribute.  
3. Each `xrow` must have an `r` attribute, indicating the row index.  
4. Each `xcell` must have both `addr` (address) and `v` (value) attributes.

Advanced validation for styles, merges and sequences will be introduced later.


In [3]:
# ============================================================
# 3. Minimal XLang validator (with type hint checking)
# ============================================================

ALLOWED_TYPES = {"number", "string", "date", "bool"}

def validate_xlang_minimal(root: ET.Element) -> list[str]:
    """
    Perform minimal validation of an XLang document.

    Checks:
      - Root tag is xworkbook
      - xsheet has name
      - xrow has r
      - xcell has addr and v
      - Optional t attributes are from an allowed set
    """
    errors: list[str] = []

    # 3.2.1 Check root tag
    if root.tag != "xworkbook":
        errors.append(f"Root tag must be 'xworkbook' but found '{root.tag}'")
        return errors

    for sheet in root.findall("xsheet"):
        # xsheet name is required
        if "name" not in sheet.attrib:
            errors.append("xsheet missing required attribute 'name'")

        # xrow must have a row index
        for xrow in sheet.findall("xrow"):
            if "r" not in xrow.attrib:
                errors.append("xrow missing required attribute 'r'")

        # xcell must specify addr and v; validate optional t
        for xcell in sheet.findall("xcell"):
            if "addr" not in xcell.attrib:
                errors.append("xcell missing required attribute 'addr'")
            if "v" not in xcell.attrib:
                errors.append("xcell missing required attribute 'v'")
            t = xcell.attrib.get("t")
            if t is not None and t not in ALLOWED_TYPES:
                errors.append(
                    f"xcell at {xcell.attrib.get('addr', '?')} has invalid type hint t='{t}'"
                )

    return errors


# 4. Minimal XLang Compiler

## 4.1 Supported Tags in This Version

The compiler implemented here supports the following tags:

1. xworkbook  
2. xsheet  
3. xrow  
4. xv  
5. xcell  

This is enough to express:

- Basic headers and data rows  
- Single-cell labels  
- Simple formulas  

## 4.2 Compilation Steps

The compilation pipeline runs as follows:

1. Parse the XLang text into an XML tree using ElementTree.  
2. Validate the basic structure with the minimal validator.  
3. Create a new Excel workbook using openpyxl.  
4. For each `xsheet`:
   1. Create a worksheet with the given name.  
   2. For each `xrow`:
      - Determine the row index and starting column.  
      - Write each `xv` value into successive columns.  
   3. For each `xcell`:
      - Write the value directly into the specified address.  
5. Save the workbook to the given path as a `.xlsx` file.

Future extensions will add support for more XLang tags and behaviours.


In [4]:
# ============================================================
# 4. Minimal XLang compiler implementation (with type inference)
# ============================================================

# ============================================================
# 4.1 Value inference helper
# ============================================================

def infer_value(raw: str, type_hint: str | None = None):
    """
    Infer the correct Python type for a cell value.

    Rules:
      - If raw starts with '=', always treat as a formula string.
      - If type_hint is provided:
          - 'number'  -> try int, then float
          - 'string'  -> keep as string
          - 'date'    -> leave as string for now (date parsing can be added later)
          - 'bool'    -> parse 'TRUE'/'FALSE' etc.
      - If no type_hint:
          - Try to parse as integer or float if it looks numeric.
          - Otherwise keep as string.
    """
    if raw is None:
        return None

    raw = str(raw)

    # Formulas must be kept as strings starting with '='
    if raw.startswith("="):
        return raw

    # Honour explicit type hint if present
    if type_hint == "string":
        return raw

    if type_hint == "bool":
        upper = raw.strip().upper()
        if upper in {"TRUE", "YES"}:
            return True
        if upper in {"FALSE", "NO"}:
            return False
        # Fall back to raw string if ambiguous
        return raw

    if type_hint == "number":
        # Try integer first, then float
        try:
            return int(raw)
        except ValueError:
            try:
                return float(raw)
            except ValueError:
                return raw  # fallback

    if type_hint == "date":
        # For now, do not parse; keep as string (you can add proper date parsing later)
        return raw

    # No type_hint: try to auto-detect number
    stripped = raw.strip()
    # Simple numeric detection: optional sign, digits, optional decimal
    import re
    if re.fullmatch(r"[+-]?\d+", stripped):
        try:
            return int(stripped)
        except ValueError:
            pass
    if re.fullmatch(r"[+-]?\d*\.\d+", stripped):
        try:
            return float(stripped)
        except ValueError:
            pass

    # Fallback: treat as plain string
    return raw
# ============================================================

def compile_xlang_to_xlsx(xlang_text: str, output_path: str | Path) -> None:
    """
    Compile a minimal subset of XLang into an Excel .xlsx file.

    Supported tags:
        - xworkbook
        - xsheet
        - xrow
        - xv
        - xcell

    Values are automatically cast:
        - Formulas: kept as strings starting with '='
        - Numeric-looking strings: converted to int or float
        - Everything else: left as string
        - Optional t attribute on xcell can force a type
    """
    # 4.2.1 Parse the XLang input
    root = ET.fromstring(xlang_text)

    # 4.2.2 Validate basic structure
    errors = validate_xlang_minimal(root)
    if errors:
        formatted = "\n".join("  - " + e for e in errors)
        raise ValueError("Invalid XLang:\n" + formatted)

    # 4.2.3 Create a new workbook and remove the default blank sheet
    wb = Workbook()
    wb.remove(wb.active)

    # 4.2.4 Process each sheet
    for xsheet in root.findall("xsheet"):
        sheet_name = xsheet.attrib["name"]
        ws = wb.create_sheet(title=sheet_name)

        # 4.2.4.1 Process xrow elements
        for xrow in xsheet.findall("xrow"):
            row_idx = int(xrow.attrib["r"])
            start_col_letter = xrow.attrib.get("c", "A")
            start_col_idx = col_letter_to_index(start_col_letter)

            for offset, xv in enumerate(xrow.findall("xv")):
                raw_value = xv.text or ""
                value = infer_value(raw_value, None)
                ws.cell(
                    row=row_idx,
                    column=start_col_idx + offset,
                    value=value
                )

        # 4.2.4.2 Process xcell elements (direct addressing)
        for xcell in xsheet.findall("xcell"):
            addr = xcell.attrib["addr"]
            raw_value = xcell.attrib["v"]
            type_hint = xcell.attrib.get("t")
            value = infer_value(raw_value, type_hint)
            ws[addr] = value

    # 4.2.5 Ensure parent directory exists and save workbook
    output_path = Path(output_path)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    wb.save(str(output_path))


# 5. Example XLang Document

## 5.1 KPI Example

This example describes a very simple worksheet named `KPI`:

1. A header row with `Region` and `Sales`.  
2. Two rows of data: `North`, `South` with numeric values.  
3. A `Total` label.  
4. A `SUM` formula that adds the sales values.

We will compile this XLang text into `kpi_example.xlsx`.


In [5]:
# ============================================================
# 5. Example XLang: KPI sheet
# ============================================================

kpi_example_xlang = """
<xworkbook>
  <xsheet name="KPI">
    <xrow r="1"><xv>Region</xv><xv>Sales</xv></xrow>
    <xrow r="2"><xv>North</xv><xv>120000</xv></xrow>
    <xrow r="3"><xv>South</xv><xv>98000</xv></xrow>
    <xcell addr="A4" v="Total"/>
    <xcell addr="B4" v="=SUM(B2:B3)"/>
  </xsheet>
</xworkbook>
""".strip()


## 5.2 Example 2 — Multi sheet regional sales with formulas

This example stresses several scenarios within the basic tag set:

1. Multiple sheets inside a single workbook.  
2. Numeric inference for positive, negative and decimal values.  
3. Formulas that reference another sheet.  
4. Mixed use of `xrow` and `xcell` in the same workbook.

Workbook structure:

Sheet "Data"  
- Row 1: headers `Region`, `Q1`, `Q2`, `Q3`  
- Rows 2 to 5: regions `North`, `South`, `East`, `West` with numeric values  
- Includes a negative number to simulate refunds  
- Includes a decimal value to test float inference  

Sheet "Summary"  
- Row 1: headers `Metric`, `Value`  
- Row 2: `TotalQ1` with formula `=SUM(Data!B2:B5)`  
- Row 3: `TotalAll` with formula `=SUM(Data!B2:D5)`  
- Row 4: `AverageQ2` with formula `=AVERAGE(Data!C2:C5)`  


In [6]:
# ============================================================
# 5.2 Example 2: Multi sheet regional sales with formulas
# ============================================================

example2_xlang = """
<xworkbook>
  <xsheet name="Data">
    <xrow r="1"><xv>Region</xv><xv>Q1</xv><xv>Q2</xv><xv>Q3</xv></xrow>
    <xrow r="2"><xv>North</xv><xv>120000</xv><xv>130000.5</xv><xv>125000</xv></xrow>
    <xrow r="3"><xv>South</xv><xv>95000</xv><xv>97000</xv><xv>99000</xv></xrow>
    <xrow r="4"><xv>East</xv><xv>88000</xv><xv>91000</xv><xv>-5000</xv></xrow>
    <xrow r="5"><xv>West</xv><xv>110000</xv><xv>115000</xv><xv>118000</xv></xrow>
  </xsheet>

  <xsheet name="Summary">
    <xrow r="1"><xv>Metric</xv><xv>Value</xv></xrow>
    <xrow r="2"><xv>TotalQ1</xv><xv>=SUM(Data!B2:B5)</xv></xrow>
    <xrow r="3"><xv>TotalAll</xv><xv>=SUM(Data!B2:D5)</xv></xrow>
    <xrow r="4"><xv>AverageQ2</xv><xv>=AVERAGE(Data!C2:C5)</xv></xrow>
  </xsheet>
</xworkbook>
""".strip()


## 5.3 Example 3 — Mixed types and non default starting columns

This example checks more complex type and layout behaviour:

1. Using the `c` attribute on `xrow` to start from a non default column.  
2. Mixed data types, including numeric IDs, string codes and booleans.  
3. Leading zero codes that must remain strings, for example `00123`.  
4. Type hints on `xcell` to enforce strings and booleans.  
5. A total row with a formula that sums a numeric column.

Workbook structure:

Sheet "MixedTypes"  
- Row 5 (starting at column B): headers `ID`, `Code`, `Flag`, `Amount`.  
- Rows 6 to 8 (starting at column B):  
  - ID: numeric values.  
  - Code: values that look numeric, but must be preserved as strings with leading zeros.  
  - Flag: boolean style values, some with YES or TRUE.  
  - Amount: decimal and negative numbers.  
- A total row at B10 and E10.  
  - B10: label `Total`.  
  - E10: formula `=SUM(E6:E8)`.


In [7]:
# ============================================================
# 5.3 Example 3: Mixed types and non default starting columns
# ============================================================

example3_xlang = """
<xworkbook>
  <xsheet name="MixedTypes">
    <xrow r="5" c="B">
      <xv>ID</xv><xv>Code</xv><xv>Flag</xv><xv>Amount</xv>
    </xrow>

    <xrow r="6" c="B">
      <xv>1</xv><xv>00123</xv><xv>TRUE</xv><xv>1000.50</xv>
    </xrow>
    <xrow r="7" c="B">
      <xv>2</xv><xv>00456</xv><xv>FALSE</xv><xv>-250.75</xv>
    </xrow>
    <xrow r="8" c="B">
      <xv>3</xv><xv>00789</xv><xv>YES</xv><xv>500</xv>
    </xrow>

    <xcell addr="C6" v="00123" t="string"/>
    <xcell addr="C7" v="00456" t="string"/>
    <xcell addr="C8" v="00789" t="string"/>

    <xcell addr="D6" v="TRUE" t="bool"/>
    <xcell addr="D7" v="FALSE" t="bool"/>
    <xcell addr="D8" v="YES" t="bool"/>

    <xcell addr="B10" v="Total"/>
    <xcell addr="E10" v="=SUM(E6:E8)"/>
  </xsheet>
</xworkbook>
""".strip()


# 6. Compile the Example Workbook

## 6.1 Compilation

The cell below compiles the example XLang text into an Excel file named `kpi_example.xlsx`.

After you run it:

1. Confirm that the file is created in your working directory.  
2. Open it in Excel to check that:
   - The sheet is called `KPI`.  
   - Cell A1 contains `Region`.  
   - Cell B1 contains `Sales`.  
   - Cell B4 contains the formula `=SUM(B2:B3)`.


In [8]:
# ============================================================
# 6. Compile the example XLang into Excel (output/ folder)
# ============================================================

output_dir = Path("output")
output_dir.mkdir(parents=True, exist_ok=True)

output_file = output_dir / "kpi_example.xlsx"

compile_xlang_to_xlsx(kpi_example_xlang, output_file)

print(f"Workbook written to: {output_file.resolve()}")


Workbook written to: C:\Users\Jackson.chai\Research\xlang\src\notebook\output\kpi_example.xlsx


## 6.2 Compile Example 2 — Regional sales workbook

This cell compiles Example 2 into an Excel file:

- Output path: `output/example2_regional_sales.xlsx`

After running this, you can open the file in Excel and check:

- The workbook has two sheets: `Data` and `Summary`.  
- Numeric cells in `Data` are stored as numbers (integers or floats).  
- Formulas in `Summary` reference the `Data` sheet correctly.


In [9]:
# ============================================================
# 6.2 Compile Example 2 into the output folder
# ============================================================

output_dir = Path("output")
output_dir.mkdir(parents=True, exist_ok=True)

example2_file = output_dir / "example2_regional_sales.xlsx"

compile_xlang_to_xlsx(example2_xlang, example2_file)

print(f"Example 2 workbook written to: {example2_file.resolve()}")


Example 2 workbook written to: C:\Users\Jackson.chai\Research\xlang\src\notebook\output\example2_regional_sales.xlsx


## 6.3 Compile Example 3 — Mixed types workbook

This cell compiles Example 3 into an Excel file:

- Output path: `output/example3_mixed_types.xlsx`

After running, inspect the file in Excel:

- Headers appear at row 5 starting from column B.  
- IDs are numeric.  
- Codes keep their leading zeros.  
- Flags behave like booleans.  
- The total formula is present at E10.


In [10]:
# ============================================================
# 6.3 Compile Example 3 into the output folder
# ============================================================

output_dir = Path("output")
output_dir.mkdir(parents=True, exist_ok=True)

example3_file = output_dir / "example3_mixed_types.xlsx"

compile_xlang_to_xlsx(example3_xlang, example3_file)

print(f"Example 3 workbook written to: {example3_file.resolve()}")


Example 3 workbook written to: C:\Users\Jackson.chai\Research\xlang\src\notebook\output\example3_mixed_types.xlsx


# 7. Programmatic Verification

## 7.1 Loading the Generated Workbook

To be confident in the compiler, it is useful to verify the contents of the generated workbook inside Python.

We expect the following:

1. A1 should be `Region`.  
2. B1 should be `Sales`.  
3. A2 should be `North`.  
4. B2 should be `120000`.  
5. A4 should be `Total`.  
6. B4 should contain the formula `=SUM(B2:B3)`.

The next cell loads the workbook with `data_only=False` so that formulas are preserved rather than evaluated.


In [11]:
# ============================================================
# 7. Inspect the generated workbook for correctness
# ============================================================

wb = load_workbook(output_file, data_only=False)
ws = wb["KPI"]

print("A1:", ws["A1"].value)
print("B1:", ws["B1"].value)
print("A2:", ws["A2"].value)
print("B2:", ws["B2"].value, type(ws["B2"].value))
print("A4:", ws["A4"].value)
print("B4 formula:", ws["B4"].value)
print("B4 data type:", ws["B4"].data_type)


A1: Region
B1: Sales
A2: North
B2: 120000 <class 'int'>
A4: Total
B4 formula: =SUM(B2:B3)
B4 data type: f


## 7.2 Verify Example 2 — Regional sales workbook

This cell performs programmatic checks on Example 2:

Checks on `Data` sheet:
1. Sheet `Data` exists.  
2. `Data!B2`, `Data!C2`, `Data!D4` are numeric types.  

Checks on `Summary` sheet:
1. Sheet `Summary` exists.  
2. Cells `Summary!B2`, `Summary!B3`, `Summary!B4` contain the expected formulas.


In [12]:
# ============================================================
# 7.2 Programmatic verification for Example 2
# ============================================================

wb2 = load_workbook(example2_file, data_only=False)

print("Example 2 sheet names:", wb2.sheetnames)

data_ws = wb2["Data"]
summary_ws = wb2["Summary"]

print("\nExample 2: Data sheet sample values and types")
print("Data!A2:", data_ws["A2"].value, type(data_ws["A2"].value))
print("Data!B2:", data_ws["B2"].value, type(data_ws["B2"].value))
print("Data!C2:", data_ws["C2"].value, type(data_ws["C2"].value))
print("Data!D4 (negative):", data_ws["D4"].value, type(data_ws["D4"].value))

print("\nExample 2: Summary sheet formulas")
print("Summary!B2:", summary_ws["B2"].value)
print("Summary!B3:", summary_ws["B3"].value)
print("Summary!B4:", summary_ws["B4"].value)


Example 2 sheet names: ['Data', 'Summary']

Example 2: Data sheet sample values and types
Data!A2: North <class 'str'>
Data!B2: 120000 <class 'int'>
Data!C2: 130000.5 <class 'float'>
Data!D4 (negative): -5000 <class 'int'>

Example 2: Summary sheet formulas
Summary!B2: =SUM(Data!B2:B5)
Summary!B3: =SUM(Data!B2:D5)
Summary!B4: =AVERAGE(Data!C2:C5)


## 7.3 Verify Example 3 — Mixed types workbook

This cell performs programmatic checks on Example 3:

Checks:
1. Headers in row 5 from column B to E are correct.  
2. ID cells are numeric (integers).  
3. Code cells are strings with leading zeros preserved.  
4. Flag cells with type hint `t="bool"` are interpreted as booleans where possible.  
5. Total row label and formula are present as expected.


In [13]:
# ============================================================
# 7.3 Programmatic verification for Example 3
# ============================================================

wb3 = load_workbook(example3_file, data_only=False)
mixed_ws = wb3["MixedTypes"]

print("Headers at row 5:")
print("B5:", mixed_ws["B5"].value)
print("C5:", mixed_ws["C5"].value)
print("D5:", mixed_ws["D5"].value)
print("E5:", mixed_ws["E5"].value)

print("\nRow 6 values and types:")
print("ID B6:", mixed_ws["B6"].value, type(mixed_ws["B6"].value))
print("Code C6:", mixed_ws["C6"].value, type(mixed_ws["C6"].value))
print("Flag D6:", mixed_ws["D6"].value, type(mixed_ws["D6"].value))
print("Amount E6:", mixed_ws["E6"].value, type(mixed_ws["E6"].value))

print("\nRow 7 values and types:")
print("ID B7:", mixed_ws["B7"].value, type(mixed_ws["B7"].value))
print("Code C7:", mixed_ws["C7"].value, type(mixed_ws["C7"].value))
print("Flag D7:", mixed_ws["D7"].value, type(mixed_ws["D7"].value))
print("Amount E7:", mixed_ws["E7"].value, type(mixed_ws["E7"].value))

print("\nRow 8 values and types:")
print("ID B8:", mixed_ws["B8"].value, type(mixed_ws["B8"].value))
print("Code C8:", mixed_ws["C8"].value, type(mixed_ws["C8"].value))
print("Flag D8:", mixed_ws["D8"].value, type(mixed_ws["D8"].value))
print("Amount E8:", mixed_ws["E8"].value, type(mixed_ws["E8"].value))

print("\nTotal row:")
print("B10 label:", mixed_ws["B10"].value)
print("E10 formula:", mixed_ws["E10"].value)


Headers at row 5:
B5: ID
C5: Code
D5: Flag
E5: Amount

Row 6 values and types:
ID B6: 1 <class 'int'>
Code C6: 00123 <class 'str'>
Flag D6: True <class 'bool'>
Amount E6: 1000.5 <class 'float'>

Row 7 values and types:
ID B7: 2 <class 'int'>
Code C7: 00456 <class 'str'>
Flag D7: False <class 'bool'>
Amount E7: -250.75 <class 'float'>

Row 8 values and types:
ID B8: 3 <class 'int'>
Code C8: 00789 <class 'str'>
Flag D8: True <class 'bool'>
Amount E8: 500 <class 'int'>

Total row:
B10 label: Total
E10 formula: =SUM(E6:E8)


# 8 Experimentation

## 8.1 Compare XLang vs Python code length

Goal:

1. Implement Example 2 and Example 3 using plain Python + openpyxl.  
2. Generate the same logical workbooks as the XLang versions.  
3. Compare:
   - Length of the XLang specification (characters).  
   - Length of the Python code required to generate the same workbook.  

Character length is a rough but useful proxy for token usage when prompting an LLM.

We will:

1. Define `build_example2_workbook` and `build_example3_workbook`.  
2. Use them to generate Excel files in the `output` folder.  
3. Use `inspect.getsource` to measure the length of the Python implementation.  
4. Compare it with `len(example2_xlang)` and `len(example3_xlang)`.


In [14]:
# ============================================================
# Plain Python implementations for Example 2 and Example 3
# ============================================================

from openpyxl import Workbook  # already imported earlier but safe to repeat


def build_example2_workbook(path: str | Path) -> None:
    wb = Workbook()

    ws_data = wb.active
    ws_data.title = "Data"

    ws_data["A1"] = "Region"
    ws_data["B1"] = "Q1"
    ws_data["C1"] = "Q2"
    ws_data["D1"] = "Q3"

    ws_data["A2"] = "North"
    ws_data["B2"] = 120000
    ws_data["C2"] = 130000.5
    ws_data["D2"] = 125000

    ws_data["A3"] = "South"
    ws_data["B3"] = 95000
    ws_data["C3"] = 97000
    ws_data["D3"] = 99000

    ws_data["A4"] = "East"
    ws_data["B4"] = 88000
    ws_data["C4"] = 91000
    ws_data["D4"] = -5000

    ws_data["A5"] = "West"
    ws_data["B5"] = 110000
    ws_data["C5"] = 115000
    ws_data["D5"] = 118000

    ws_summary = wb.create_sheet(title="Summary")

    ws_summary["A1"] = "Metric"
    ws_summary["B1"] = "Value"

    ws_summary["A2"] = "TotalQ1"
    ws_summary["B2"] = "=SUM(Data!B2:B5)"

    ws_summary["A3"] = "TotalAll"
    ws_summary["B3"] = "=SUM(Data!B2:D5)"

    ws_summary["A4"] = "AverageQ2"
    ws_summary["B4"] = "=AVERAGE(Data!C2:C5)"

    wb.save(str(path))


def build_example3_workbook(path: str | Path) -> None:
    wb = Workbook()
    ws = wb.active
    ws.title = "MixedTypes"

    ws["B5"] = "ID"
    ws["C5"] = "Code"
    ws["D5"] = "Flag"
    ws["E5"] = "Amount"

    ws["B6"] = 1
    ws["C6"] = "00123"
    ws["D6"] = True
    ws["E6"] = 1000.50

    ws["B7"] = 2
    ws["C7"] = "00456"
    ws["D7"] = False
    ws["E7"] = -250.75

    ws["B8"] = 3
    ws["C8"] = "00789"
    ws["D8"] = True
    ws["E8"] = 500

    ws["B10"] = "Total"
    ws["E10"] = "=SUM(E6:E8)"

    wb.save(str(path))


In [15]:
# ============================================================
# Build Example 2 and Example 3 using plain Python
# ============================================================

output_dir = Path("output")
output_dir.mkdir(parents=True, exist_ok=True)

example2_python_file = output_dir / "example2_python.xlsx"
example3_python_file = output_dir / "example3_python.xlsx"

build_example2_workbook(example2_python_file)
build_example3_workbook(example3_python_file)

print(f"Python-built Example 2 workbook: {example2_python_file.resolve()}")
print(f"Python-built Example 3 workbook: {example3_python_file.resolve()}")


Python-built Example 2 workbook: C:\Users\Jackson.chai\Research\xlang\src\notebook\output\example2_python.xlsx
Python-built Example 3 workbook: C:\Users\Jackson.chai\Research\xlang\src\notebook\output\example3_python.xlsx


## 8.2 Compare XLang length vs Python code length

We now compare:

1. Length of the XLang specification (number of characters in the XLang string).  
2. Length of the Python implementation (number of characters in the source code of the builder function, plus the import line).

This is not a perfect token count, but it is a reasonable proxy because:
- LLM token counts are roughly proportional to character counts.  
- Longer code means more tokens to generate, read or send as tools.

We will use `inspect.getsource` to retrieve the function source code exactly as written in this notebook.


In [16]:
# ============================================================
# Compute and print length comparison
# ============================================================

import inspect

# Build source strings for the Python implementations, including a minimal import
example2_python_source = "from openpyxl import Workbook\n\n" + inspect.getsource(build_example2_workbook)
example3_python_source = "from openpyxl import Workbook\n\n" + inspect.getsource(build_example3_workbook)

# Character lengths
len_example2_xlang = len(example2_xlang)
len_example3_xlang = len(example3_xlang)

len_example2_py = len(example2_python_source)
len_example3_py = len(example3_python_source)

print("Example 2:")
print("  XLang length (chars): ", len_example2_xlang)
print("  Python length (chars):", len_example2_py)
print("  Python / XLang ratio: ", round(len_example2_py / len_example2_xlang, 3))

print("\nExample 3:")
print("  XLang length (chars): ", len_example3_xlang)
print("  Python length (chars):", len_example3_py)
print("  Python / XLang ratio: ", round(len_example3_py / len_example3_xlang, 3))


Example 2:
  XLang length (chars):  751
  Python length (chars): 1071
  Python / XLang ratio:  1.426

Example 3:
  XLang length (chars):  773
  Python length (chars): 570
  Python / XLang ratio:  0.737


## 8.3 Interpretation of Results

The character length comparison (excluding comments from the Python code) is:

Example 2  
XLang length: 751 characters  
Python length: 1071 characters  
Python to XLang ratio: approximately 1.43  

Example 3  
XLang length: 773 characters  
Python length: 570 characters  
Python to XLang ratio: approximately 0.74  

---

### 8.3.1 What the ratios show

These measurements give a clearer and more balanced view of XLang’s practical compression properties.

For Example 2, XLang remains significantly more concise. The multi row structure, multiple numeric columns and clear table layout are well aligned with XLang’s row-driven syntax. Python requires explicit cell references and worksheet interactions, making it inherently longer. A factor of roughly 1.43 in favour of XLang indicates that v1 already achieves meaningful compression for structured grid based workloads.

For Example 3, Python is shorter. This result is instructive. The example emphasises irregular constructs such as mixed type cells, leading zeros, explicit single cell overrides and starting columns other than A. These details force XLang v1 to rely heavily on `xcell` elements, which eliminate its natural row based compression. In contrast, direct Python assignments remain compact when dealing with sparse or irregular layouts. A ratio below 1.0 indicates that XLang v1 is not yet optimal for override heavy or non tabular structures.

---

### 8.3.2 Interpretation for language viability

The results sharpen the picture of where XLang is most effective.

XLang compresses well when:

1. Data aligns with a predictable table format  
2. Rows contain several adjacent values  
3. Sheet layout is mostly regular  
4. Few explicit overrides are required  

XLang is less effective when:

1. Many cells must be individually specified  
2. Leading zeros or mixed types require explicit tagging  
3. Layout begins at unconventional ranges  
4. Pattern irregularity dominates the sheet structure  

This matches expectations for a v1 language that prioritises clarity and safety over aggressive shorthand.

---

### 8.3.3 Implications for ORO and future design

The results do not weaken the case for XLang; instead, they clarify the direction for future development.

The findings indicate:

1. XLang v1 already improves conciseness in the types of spreadsheets that dominate analytics, reporting and LLM generated dashboards.  
2. Compression collapses when forced into scenarios that lack structural regularity.  
3. To expand XLang’s advantage, higher level constructs are required, such as:
   - `xrange` for multi cell spans  
   - `xrepeat` for pattern replication  
   - `xseq` and `xplace` for generative table structures  
   - `xstyle` and `xmerge` for merged and styled shorthand  
4. These constructs would turn many explicit overrides in Example 3 into short declarative expressions, reversing the ratio in favour of XLang even in irregular cases.

The experiment therefore reinforces the viability of XLang as a foundation for Output Representation Optimisation, while highlighting exactly where compression gains can be unlocked in v2 and v3 of the language.
